question 4 - least squares

In [23]:
import numpy as np
b = np.array([6.0,1,5,2])
A = np.array([[2.0,1,2],[1,-2,1],[1,2,3],[1,1,1]])
ATA = A.transpose()@A

In [24]:
def forward_sub(L,b):
    L = np.copy(L)
    b = np.copy(b)
    for i in range(len(L)):
        b[i] = b[i] / L[i,i]
        L[i][i] = 1
        for j in range(i+1,len(L)):
            b[j] = b[j] - b[i] * (L[j][i])
            L[j] = L[j] - L[i] * (L[j][i])
    return b


In [25]:
def backward_sub(LT,y):
    LT = np.copy(LT)
    y = np.copy(y)
    for i in reversed(range(len(LT))):
        y[i] = y[i] / LT[i,i]
        LT[i][i] = 1
        for j in range(0,i):
            y[j] = y[j] - y[i] * (LT[j][i])
            LT[j] = LT[j] - LT[i] * (LT[j][i])
    return y

Section (a) - using Cholesky Factorization

In [26]:
ATB = A.transpose() @ b
L = np.linalg.cholesky(ATA)
y = forward_sub(L,ATB)
x = backward_sub(L.transpose(),y)
print(f'the vector that we found is x={x}')




the vector that we found is x=[1.7 0.6 0.7]


Section (b) using QR , SVD

In [29]:
Q,R = np.linalg.qr(A)
y = forward_sub(R.transpose(),ATB)
x_qr = backward_sub(R,y)
print(f'The vector that we found using qr factorization is x={x_qr}')


The vector that we found is x=[1.7 0.6 0.7]
[0.00000000e+00 3.55271368e-15 0.00000000e+00]
